--- Day 20: Particle Swarm ---

Suddenly, the GPU contacts you, asking for help. Someone has asked it to simulate too many particles, and it won't be able to finish them all in time to render the next frame at this rate.

It transmits to you a buffer (your puzzle input) listing each particle in order (starting with particle 0, then particle 1, particle 2, and so on). For each particle, it provides the X, Y, and Z coordinates for the particle's position (p), velocity (v), and acceleration (a), each in the format <X,Y,Z>.

Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:

```
Increase the X velocity by the X acceleration.
Increase the Y velocity by the Y acceleration.
Increase the Z velocity by the Z acceleration.
Increase the X position by the X velocity.
Increase the Y position by the Y velocity.
Increase the Z position by the Z velocity.
```

Because of seemingly tenuous rationale involving z-buffering, the GPU would like to know which particle will stay closest to position `<0,0,0>` in the long term. Measure this using the Manhattan distance, which in this situation is simply the sum of the absolute values of a particle's X, Y, and Z position.

For example, suppose you are only given two particles, both of which stay entirely on the X-axis (for simplicity). Drawing the current states of particles 0 and 1 (in that order) with an adjacent a number line and diagram of current X positions (marked in parenthesis), the following would take place:

```
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>                         (0)(1)

p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>                      (1)   (0)

p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>          (1)               (0)

p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>                         (0)   
```

At this point, particle 1 will never be closer to <0,0,0> than particle 0, and so, in the long run, particle 0 will stay closest.

Which particle will stay closest to position <0,0,0> in the long term?

In [1]:
test_particles = [
    "p=<3,0,0>, v=<2,0,0>, a=<-1,0,0>",
    "p=<4,0,0>, v=<0,0,0>, a=<-2,0,0>"
]

In [2]:
def particle_struct(particle):
    components = particle.split(", ")
    position = [int(x) for x in components[0].split("=")[1].strip("<").strip(">").split(",")]
    velocity = [int(x) for x in components[1].split("=")[1].strip("<").strip(">").split(",")]
    accel = [int(x) for x in components[2].split("=")[1].strip("<").strip(">").split(",")]
    return {"p":position, "v":velocity, "a":accel}

In [3]:
def update_particle(particle):
    for i in range(3):
        particle['v'][i] += particle['a'][i]
    for j in range(3):
        particle['p'][j] += particle['v'][j]
    particle['d'] = sum([abs(x) for x in particle['p']])
    return particle

In [4]:
test_particles = [particle_struct(x) for x in test_particles]
test_particles

[{'p': [3, 0, 0], 'v': [2, 0, 0], 'a': [-1, 0, 0]},
 {'p': [4, 0, 0], 'v': [0, 0, 0], 'a': [-2, 0, 0]}]

In [5]:
for i in range(3):
    print("Step:", i)
    [update_particle(x) for x in test_particles]
    print(test_particles)

Step: 0
[{'p': [4, 0, 0], 'v': [1, 0, 0], 'a': [-1, 0, 0], 'd': 4}, {'p': [2, 0, 0], 'v': [-2, 0, 0], 'a': [-2, 0, 0], 'd': 2}]
Step: 1
[{'p': [4, 0, 0], 'v': [0, 0, 0], 'a': [-1, 0, 0], 'd': 4}, {'p': [-2, 0, 0], 'v': [-4, 0, 0], 'a': [-2, 0, 0], 'd': 2}]
Step: 2
[{'p': [3, 0, 0], 'v': [-1, 0, 0], 'a': [-1, 0, 0], 'd': 3}, {'p': [-8, 0, 0], 'v': [-6, 0, 0], 'a': [-2, 0, 0], 'd': 8}]


In [6]:
with open("input.txt") as f:
    particle_file = f.read().split("\n")

In [7]:
particles = [particle_struct(x) for x in particle_file]

In [16]:
particles[0]

{'p': [2011979, 4526094, -71572],
 'v': [4011, 9024, -73],
 'a': [4, 9, 0],
 'd': 6609645}

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(particles)
df.head(10)

,p,v,a
0,"[-1021, -2406, 1428]","[11, 24, -73]","[4, 9, 0]"
1,"[-181, -26, 548]","[11, 73, -8]","[0, -7, -2]"
2,"[-2101, -2706, -1112]","[-19, -66, 75]","[12, 19, -2]"
3,"[-1221, -16, -1412]","[0, -1, -15]","[6, 0, 8]"
4,"[1009, -716, -1062]","[-164, 55, 125]","[11, -2, -7]"
5,"[-671, 1794, -1062]","[46, 87, 41]","[-1, -17, 1]"
6,"[255, -544, 641]","[-21, 93, -83]","[-1, -7, 4]"
7,"[-945, -1004, -29]","[88, 106, -38]","[1, -1, 8]"
8,"[-1045, -644, -949]","[76, 114, 98]","[5, -9, 0]"
9,"[640, 956, 741]","[-21, -79, -49]","[-8, -3, -4]"


In [10]:
def manhattan_dist(some_list):
    abs_list = [abs(x) for x in some_list]
    return(sum(abs_list))

In [11]:
df['d'] = df['p'].map(manhattan_dist)
df.head(5)

,p,v,a,d
0,"[-1021, -2406, 1428]","[11, 24, -73]","[4, 9, 0]",4855
1,"[-181, -26, 548]","[11, 73, -8]","[0, -7, -2]",755
2,"[-2101, -2706, -1112]","[-19, -66, 75]","[12, 19, -2]",5919
3,"[-1221, -16, -1412]","[0, -1, -15]","[6, 0, 8]",2649
4,"[1009, -716, -1062]","[-164, 55, 125]","[11, -2, -7]",2787


In [12]:
for i in range(1000):
    [update_particle(x) for x in particles]

In [13]:
df1000 = pd.DataFrame(particles)

In [14]:
df1000.dtypes

p    object
v    object
a    object
d     int64
dtype: object

In [17]:
df1000.loc[df1000.d == df1000.d.min()].index

Int64Index([144], dtype='int64')

The first half of this puzzle is complete! It provides one gold star: *

--- Part Two ---

To simplify the problem further, the GPU would like to remove any particles that **collide**. Particles collide if their positions ever **exactly match**. Because particles are updated simultaneously, **more than two particles** can collide at the same time and place. Once particles collide, they are removed and cannot collide with anything else after that tick.

For example:
```
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>    (0)   (1)   (2)            (3)
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=<-3,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-2,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-1,0,0>, v=< 1,0,0>, a=< 0,0,0>             (0)(1)(2)      (3)   
p=< 2,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=< 0,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=< 0,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=< 0,0,0>, v=< 1,0,0>, a=< 0,0,0>                       X (3)      
p=< 1,0,0>, v=<-1,0,0>, a=< 0,0,0>

------destroyed by collision------    
------destroyed by collision------    -6 -5 -4 -3 -2 -1  0  1  2  3
------destroyed by collision------                      (3)         
p=< 0,0,0>, v=<-1,0,0>, a=< 0,0,0>
```

In this example, particles 0, 1, and 2 are simultaneously destroyed at the time and place marked X. On the next tick, particle 3 passes through unharmed.

How many particles are left after all collisions are resolved?

In [18]:
particles = [particle_struct(x) for x in particle_file]

In [19]:
particle_df = pd.DataFrame(particles)

In [20]:
particle_df['position'] = particle_df.p.apply(str)

In [21]:
particle_df.head()

,p,v,a,position
0,"[-1021, -2406, 1428]","[11, 24, -73]","[4, 9, 0]","[-1021, -2406, 1428]"
1,"[-181, -26, 548]","[11, 73, -8]","[0, -7, -2]","[-181, -26, 548]"
2,"[-2101, -2706, -1112]","[-19, -66, 75]","[12, 19, -2]","[-2101, -2706, -1112]"
3,"[-1221, -16, -1412]","[0, -1, -15]","[6, 0, 8]","[-1221, -16, -1412]"
4,"[1009, -716, -1062]","[-164, 55, 125]","[11, -2, -7]","[1009, -716, -1062]"


In [22]:
particle_df.groupby('position').count() > 1

,p,v,a
position,,,
"[-10007, -68, 2480]",False,False,False
"[-1001, -2217, 1954]",False,False,False
"[-1002, 2554, -1159]",False,False,False
"[-1011, 2553, 1237]",False,False,False
"[-102, -412, 1258]",False,False,False
...,...,...,...
"[970, -172, 991]",False,False,False
"[9714, -2167, -1770]",False,False,False
"[9892, -2332, -8660]",False,False,False


In [23]:
positions = [p.get('p') for p in particles]
positions[0:10]

[[-1021, -2406, 1428],
 [-181, -26, 548],
 [-2101, -2706, -1112],
 [-1221, -16, -1412],
 [1009, -716, -1062],
 [-671, 1794, -1062],
 [255, -544, 641],
 [-945, -1004, -29],
 [-1045, -644, -949],
 [640, 956, 741]]

In [24]:
def find_collisions(particles):
    dup_pos = []
    positions = [p.get('p') for p in particles]
    for i in range(len(positions)):
        remain_pos = positions[:i] + positions[i+1:]
        if  positions[i] in remain_pos and positions[i] not in dup_pos:
            dup_pos.append(positions[i])
    return dup_pos

In [25]:
find_collisions([{'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,1,0]}, {'p':[0,1,0]}, {'p':[0,0,1]}])

[[0, 0, 0], [0, 1, 0]]

In [26]:
def remove_collisions(particles, collisions):
    particles = [p for p in particles if p.get('p') not in collisions]
    return particles

In [27]:
remove_collisions([{'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,1,0]}, {'p':[0,1,0]}, {'p':[0,0,1]}],
                 find_collisions([{'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,1,0]}, {'p':[0,1,0]}, {'p':[0,0,1]}]))

[{'p': [0, 0, 1]}]

In [28]:
pt_2_tests = [
    "p=<-6,0,0>, v=<3,0,0>, a=<0,0,0>",
    "p=<-4,0,0>, v=<2,0,0>, a=<0,0,0>",
    "p=<-2,0,0>, v=<1,0,0>, a=<0,0,0>",
    "p=<3,0,0>, v=<-1,0,0>, a=<0,0,0>",
]

pt2_tests = [particle_struct(x) for x in pt_2_tests]
pt2_tests

[{'p': [-6, 0, 0], 'v': [3, 0, 0], 'a': [0, 0, 0]},
 {'p': [-4, 0, 0], 'v': [2, 0, 0], 'a': [0, 0, 0]},
 {'p': [-2, 0, 0], 'v': [1, 0, 0], 'a': [0, 0, 0]},
 {'p': [3, 0, 0], 'v': [-1, 0, 0], 'a': [0, 0, 0]}]

In [29]:
pt2_test_particles = [x for x in pt2_tests]
for i in range(3):
    pt2_test_particles = [update_particle(x) for x in pt2_test_particles]
    collisions = find_collisions(pt2_test_particles)
    pt2_test_particles = remove_collisions(pt2_test_particles, collisions)
print("\nFinal:", len(pt2_test_particles))


Final: 1


In [30]:
updated_particles = [particle_struct(x) for x in particle_file]
cur_len = len(updated_particles)
for i in range(1000):
    updated_particles = [update_particle(x) for x in updated_particles]
    collisions = find_collisions(updated_particles)
    updated_particles = remove_collisions(updated_particles, collisions)
    if len(updated_particles) < cur_len:
        cur_len = len(updated_particles)
        print("After", i, ":", cur_len)

After 9 : 992
After 10 : 974
After 11 : 957
After 12 : 926
After 13 : 901
After 14 : 881
After 15 : 858
After 16 : 813
After 17 : 780
After 18 : 775
After 19 : 754
After 21 : 736
After 22 : 726
After 23 : 701
After 24 : 686
After 25 : 674
After 26 : 671
After 27 : 652
After 28 : 648
After 29 : 619
After 30 : 607
After 31 : 583
After 32 : 574
After 33 : 560
After 34 : 549
After 35 : 538
After 36 : 532
After 37 : 503
After 38 : 477


That's the right answer! You are one gold star closer to debugging the printer. You got rank 848 on this star's leaderboard.
